In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['Arial Unicode MS']
mpl.rcParams['axes.unicode_minus']=False
import yfinance as yf

# Define scoring functions
def cal_tryoy(value):
    # Example logic: return a score based on tr_yoy
    if value > 10:
        return 10
    elif value > 5:
        return 5
    else:
        return 0

def cal_opyoy(value):
    # Example logic: return a score based on op_yoy
    if value > 20:
        return 10
    elif value > 10:
        return 5
    else:
        return 0

def cal_gpm(value):
    # Example logic: return a score based on grossprofit_margin
    if value > 30:
        return 10
    elif value > 20:
        return 5
    else:
        return 0

def cal_exp(value):
    # Example logic: return a score based on expense_of_sales
    if value < 10:
        return 10
    elif value < 15:
        return 5
    else:
        return 0

def cal_inv(value):
    # Example logic: return a score based on inv_turn
    if value > 2:
        return 10
    elif value > 1:
        return 5
    else:
        return 0

def cal_ocfp(value):
    # Example logic: return a score based on ocfps
    if value > 3:
        return 10
    elif value > 1:
        return 5
    else:
        return 0

def cal_roe(value):
    # Example logic: return a score based on roe_yearly
    if value > 15:
        return 10
    elif value > 10:
        return 5
    else:
        return 0

def cal_roa(value):
    # Example logic: return a score based on roa2_yearly
    if value > 8:
        return 10
    elif value > 5:
        return 5
    else:
        return 0

def cal_pb(value):
    # Example logic: return a score based on pb
    if value < 2:
        return 10
    elif value < 3:
        return 5
    else:
        return 0

def cal_pe(value):
    # Example logic: return a score based on pe_ttm
    if value < 15:
        return 10
    elif value < 25:
        return 5
    else:
        return 0
# Mock get_indicators function for testing (Replace this with your actual implementation)
def get_indicators(code):
    # Mock data for testing
    data = pd.DataFrame({
        'tr_yoy': np.random.uniform(5, 15, 10),  # 营业收入增长率
        'op_yoy': np.random.uniform(10, 30, 10),  # 营业利润增长率
        'grossprofit_margin': np.random.uniform(20, 40, 10),  # 毛利率
        'expense_of_sales': np.random.uniform(5, 15, 10),  # 期间费用率
        'inv_turn': np.random.uniform(1, 3, 10),  # 存货周转率
        'ocfps': np.random.uniform(0, 5, 10),  # 每股经营性现金流
        'eps': np.random.uniform(0, 5, 10),  # 每股收益
        'roe_yearly': np.random.uniform(10, 20, 10),  # 年化净资产收益率
        'roa2_yearly': np.random.uniform(5, 10, 10),  # 总资产收益率
        'pb': np.random.uniform(1, 5, 10),  # 市净率
        'pe_ttm': np.random.uniform(10, 30, 10),  # 动态市盈率
        'netprofit_yoy': np.random.uniform(10, 20, 10)  # 盈利增长率
    })
    return data

def indicator_score(code):
    data = get_indicators(code)
    
    # Add date index if not present
    if 'date' not in data.columns:
        data['date'] = pd.date_range(start='2020-01-01', periods=len(data), freq='D')

    # Define a list of all required columns
    required_columns = ['tr_yoy', 'op_yoy', 'grossprofit_margin', 'expense_of_sales', 'inv_turn', 
                        'ocfps', 'eps', 'roe_yearly', 'roa2_yearly', 'pb', 'pe_ttm', 'netprofit_yoy']

    # Check if all required columns are present
    for col in required_columns:
        if col not in data.columns:
            raise KeyError(f"Required column '{col}' not found in data")

    # Calculate scores
    data['营收得分'] = data['tr_yoy'].apply(cal_tryoy)
    data['利润得分'] = data['op_yoy'].apply(cal_opyoy)
    data['gpm'] = data['grossprofit_margin'] - data['grossprofit_margin'].rolling(3).mean()
    data['毛利得分'] = data['gpm'].apply(cal_gpm)
    data['exp'] = data['expense_of_sales'] - data['expense_of_sales'].rolling(3).mean()
    data['费用得分'] = data['exp'].apply(cal_exp)
    data['inv'] = (data['inv_turn'] - data['inv_turn'].rolling(3).mean()) * 100 / data['inv_turn'].rolling(3).mean()
    data['周转得分'] = data['inv'].apply(cal_inv)
    data['ocf'] = (data['ocfps'].rolling(3).sum() - data['eps'].rolling(3).sum()) * 100 / data['eps'].rolling(3).sum()
    data['现金得分'] = data['ocf'].apply(cal_ocfp)
    data['净资产得分'] = data['roe_yearly'].apply(cal_roe)
    data['总资产得分'] = data['roa2_yearly'].apply(cal_roa)
    data['市净率得分'] = data['pb'].apply(cal_pb)
    data['peg'] = data['pe_ttm'] / data['netprofit_yoy'].rolling(3).mean()
    data['市盈率得分'] = data['peg'].apply(cal_pe)

    # Calculate total score
    data['总分'] = data[['营收得分', '利润得分', '毛利得分', '费用得分', '周转得分', '现金得分', '净资产得分', 
                    '总资产得分', '市净率得分', '市盈率得分']].sum(axis=1)

    # Return the final DataFrame
    return data[['date', '营收得分', '利润得分', '毛利得分', '费用得分', '周转得分', '现金得分', '净资产得分', 
                 '总资产得分', '市净率得分', '市盈率得分', '总分']]

# Test the function
code = '2330.TW'
result = indicator_score(code)
print(result)

        date  营收得分  利润得分  毛利得分  费用得分  周转得分  现金得分  净资产得分  总资产得分  市净率得分  市盈率得分  \
0 2020-01-01    10    10     0     0     0     0     10      5     10      0   
1 2020-01-02    10    10     0     0     0     0      5     10     10      0   
2 2020-01-03     5    10     0    10    10     0      5      5      5     10   
3 2020-01-04    10     5     0    10     0     0     10      5      0     10   
4 2020-01-05     5    10     0    10     0    10     10     10     10     10   
5 2020-01-06    10    10     0    10    10    10      5     10     10     10   
6 2020-01-07    10     5     0    10     0    10      5      5      5     10   
7 2020-01-08     5     5     0    10    10     0     10      5      0     10   
8 2020-01-09    10     5     0    10     0     0     10     10      0     10   
9 2020-01-10     5    10     0    10    10     0     10     10      0     10   

   总分  
0  45  
1  45  
2  60  
3  50  
4  75  
5  85  
6  60  
7  55  
8  55  
9  65  
